In [76]:
import pandas as pd
import numpy as np

In [77]:
df = pd.read_csv('data.csv')

# Embedding với Sentence-BERT

In [79]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load Sentence-BERT

In [80]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Ghép các trường lại thành 1 text để embedding

In [81]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3896 entries, 0 to 3895
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HotelID       3896 non-null   int64  
 1   Name Hotel    3896 non-null   object 
 2   Address       3896 non-null   object 
 3   Location      3896 non-null   object 
 4   Descriptions  3896 non-null   object 
 5   Rating        3896 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 182.8+ KB
None


In [84]:
df["TextForEmbedding"] = (
    df["Name Hotel"].astype(str).str.lower() + " " +
    df["Address"].astype(str).str.lower() + " " +
    df["Location"].astype(str).str.lower() + " " +
    df["Descriptions"].astype(str).str.lower()
)


# Sinh embedding cho toàn bộ dataset

In [85]:
embeddings = sbert_model.encode(df["TextForEmbedding"].tolist(), batch_size=32, show_progress_bar=True)

Batches:   0%|          | 0/122 [00:00<?, ?it/s]

In [86]:
print("Embedding shape:", embeddings.shape)   # (số dòng, 384)

Embedding shape: (3896, 384)


In [88]:
print("✅ Embedding done!")
print("Kiểu dữ liệu:", type(embeddings))     # numpy.ndarray
print("Shape:", embeddings.shape)            # (số_samples, số_chiều) ví dụ (1000, 384)
print("Dtype:", embeddings.dtype)            # float32 hoặc float64

# In thử vector của 1 khách sạn đầu tiên
print("Vector mẫu (10 phần tử đầu):", embeddings[0][:384])


✅ Embedding done!
Kiểu dữ liệu: <class 'numpy.ndarray'>
Shape: (3896, 384)
Dtype: float32
Vector mẫu (10 phần tử đầu): [ 3.61901671e-01  6.19400293e-03  1.00559674e-01  1.24107383e-01
  1.44469410e-01 -5.66059016e-02 -1.53053463e-01 -1.97330087e-01
 -1.37990668e-01  2.73341030e-01  9.36376005e-02 -2.69652426e-01
  1.14715975e-02  2.39548236e-01  1.76534116e-01 -6.22116355e-03
  1.46392480e-01  1.22105703e-01  3.61636251e-01  1.21856309e-01
  5.08252233e-02  1.03973616e-02 -1.57167643e-01  1.88926142e-02
  2.07447007e-01  1.51986275e-02  1.58515930e-01  2.33707115e-01
 -1.30333483e-01 -1.89318880e-02  1.23486519e-01  4.29721400e-02
 -1.98622718e-01  1.00280575e-01  1.81054458e-01  1.16698071e-01
 -1.67736225e-02 -1.51439369e-01 -2.05556363e-01 -8.82036313e-02
 -1.13890390e-03 -3.09397075e-02  3.35850894e-01 -1.47777488e-02
 -1.16681596e-02  2.08786763e-02 -9.71937254e-02 -7.41432048e-03
  1.85174286e-01 -1.93408117e-01 -1.13190144e-01 -1.45647243e-01
 -1.11614719e-01 -3.00235767e-02  4.

# Lưu vào Milvus

In [89]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection, utility, connections

# Kết nối đến Milvus server

In [90]:
connections.connect(alias="default",host="localhost",port="19530")

# Tạo collection schema

In [91]:
collection_name = "hotels_collection"
 # Kiểm tra xem collection đã tồn tại chưa
if not utility.has_collection(collection_name):
    print(f"🚀 Collection '{collection_name}' chưa tồn tại, tạo mới...")

    fields = [
        FieldSchema(name="HotelID", dtype=DataType.INT64, is_primary=True, auto_id=False),
        FieldSchema(name="TextForEmbedding", dtype=DataType.FLOAT_VECTOR, dim=384),  # Sentence-BERT = 384, PhoBERT = 768
    ]
    schema = CollectionSchema(fields=fields, description="Hotel dataset embeddings")

    collection = Collection(name=collection_name, schema=schema)
else:
    print(f"✅ Collection '{collection_name}' đã tồn tại.")
    collection = Collection(collection_name)

🚀 Collection 'hotels_collection' chưa tồn tại, tạo mới...


# Chuẩn bị dữ liệu để insert

In [92]:
hotel_ids = df["HotelID"].astype(int).tolist()

In [93]:
print(type(embeddings))   

<class 'numpy.ndarray'>


# Insert theo batch

In [94]:
 collection.insert([
        batch_ids,
        batch_embeddings.tolist(),
    ])

collection.flush()
print(f"📊 Số lượng bản ghi: {collection.num_entities}")

📊 Số lượng bản ghi: 3896


In [95]:
index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",     
    "params": {"nlist": 1024}
}

# Tạo index cho field vector
try:
    if not collection.has_index(field_name="TextForEmbedding"):
        print("🚀 Đang tạo index cho vector...")
        collection.create_index(field_name="TextForEmbedding", index_params=index_params)
        print("✅ Index đã được tạo thành công!")
    else:
        print("✅ Index đã tồn tại cho 'TextForEmbedding'.")
except Exception as e:
    print(f"❌ Lỗi khi kiểm tra hoặc tạo index: {e}")


🚀 Đang tạo index cho vector...
✅ Index đã được tạo thành công!


In [96]:
collection.load()

# demo

In [98]:
sentences = ["tp.hồ chí minh"]

# Tạo embedding
embeddings_test = sbert_model.encode(sentences)

print("Sentence-BERT embedding shape:", embeddings_test.shape)
print(embeddings_test.tolist())

Sentence-BERT embedding shape: (1, 384)
[[0.09211999177932739, 0.10914507508277893, 0.09477809071540833, -0.009518093429505825, -0.10266107320785522, -0.010812096297740936, 0.18513241410255432, 0.18567846715450287, 0.16354164481163025, -0.05412594601511955, 0.043132394552230835, -0.2314421385526657, 0.03095582313835621, -0.009989823214709759, -0.09178463369607925, -0.1113564521074295, 0.13538552820682526, -0.013828140683472157, -0.18668264150619507, -0.03113013133406639, -0.004867078270763159, -0.009879546239972115, -0.026808688417077065, 0.023760642856359482, 0.12369603663682938, -0.0776580274105072, -0.027718933299183846, -0.049692556262016296, -0.051368873566389084, -0.15256719291210175, 0.04201161861419678, -0.06799226999282837, 0.17071637511253357, 0.14306500554084778, -0.04160996526479721, 0.1484261006116867, -0.0052887797355651855, 0.09730474650859833, 0.06823854893445969, 0.016386035829782486, 0.0027221953496336937, -0.049188584089279175, 0.0702298954129219, 0.28886082768440247